# Модуль 2.1 — SequentialChain / LCEL

**Цель:** собрать простой пайплайн: очистка → суммаризация → классификация.

**Что сделаем:**
- установим библиотеки
- настроим ключ и base URL
- соберем цепочку через LCEL (`RunnableSequence`)
- разберем параметры и практические советы

---
### 🎯 Якорь
Цепочка = конвейер: помыл → нарезал → подал. Выход одного шага = вход следующего.

**Термины:** Chain | LCEL (`|`) | invoke

In [ ]:
%pip -q install -U \
  langchain \
  langchain-openai \
  langchain-community \
  pydantic==2.12.3 \
  python-dotenv \
  tiktoken \
  requests==2.32.4

## Настройка ключа и base URL

Для AITunnel укажите `OPENAI_API_KEY` и `OPENAI_BASE_URL`. В Colab лучше хранить ключ в переменной окружения.

> Подсказка: можно задать ключ один раз в разделе **Secrets** (значок 🔑 слева в Colab), тогда он будет доступен во всех ячейках.

In [ ]:
import os
from getpass import getpass
from dotenv import load_dotenv

load_dotenv()

if not os.environ.get("OPENAI_API_KEY"):
    os.environ["OPENAI_API_KEY"] = getpass("Введите OPENAI_API_KEY: ")

if not os.environ.get("OPENAI_BASE_URL"):
    os.environ["OPENAI_BASE_URL"] = "https://api.aitunnel.ru/v1/"

## LCEL: цепочка из трех шагов

Мы сделаем простой ETL‑пайплайн:
1. **Очистка** (убираем шум)
2. **Суммаризация**
3. **Классификация** (определяем тему)

LCEL (LangChain Expression Language) позволяет собирать цепочки через оператор `|`.

## Разбор пайплайна: зачем и как работает каждый шаг

### 1) Очистка (cleaning)
**Зачем:**
- убираем повторы, лишние эмоции, мусорные символы
- повышаем качество следующего шага (суммаризация становится точнее)

**Как выполняется:**
- модель получает инструкцию «очистить» и вернуть тот же смысл
- важно сохранить факты и эмоции, но убрать шум

**Типичные правила:**
- убрать дубли и лишние восклицания
- сохранить ключевые факты (доставка, товар, оценка)

---

### 2) Суммаризация (summarization)
**Зачем:**
- сокращаем текст до сути
- уменьшаем объем данных для дальнейшей обработки

**Как выполняется:**
- просим 2–3 предложения с ключевыми фактами
- задаем лимит длины, чтобы не растекаться

**Типичные правила:**
- 1–2 ключевые проблемы + общий вывод
- без новых фактов

---

### 3) Классификация (classification)
**Зачем:**
- приводим текст к простому тегу, который удобно использовать в аналитике
- позволяет строить отчеты и маршрутизацию (например, «логистика», «качество») 

**Как выполняется:**
- просим одно слово или короткий тег
- можно задать список допустимых классов

**Типичные правила:**
- одна тема = один тег
- если сомнение — выбираем наиболее важную проблему

---

### Почему порядок именно такой
- очистка → суммаризация: меньше шума = лучше выжимка
- суммаризация → классификация: классифицируем уже «суть», а не весь текст

### Практические советы
- Для каждого шага держите **низкую температуру** (0.1–0.3), чтобы не искажать смысл.
- Добавляйте **ограничения формата** (длина, стиль, один тег).
- Логируйте промежуточные результаты — это ускоряет отладку.

In [ ]:
from langchain_openai import ChatOpenAI
from langchain.prompts import PromptTemplate
from langchain_core.output_parsers import StrOutputParser

base_url = os.environ.get("OPENAI_BASE_URL")
chat = ChatOpenAI(
    model="gpt-5.2-chat",
    temperature=0.2,
    max_tokens=256,
    base_url=base_url,
)

clean_prompt = PromptTemplate(
    input_variables=["text"],
    template=(
        "Очисти текст от шума и повторов, сохрани смысл.\n"
        "Текст: {text}"
    ),
)

summarize_prompt = PromptTemplate(
    input_variables=["text"],
    template=(
        "Сделай краткую выжимку в 2-3 предложения.\n"
        "Текст: {text}"
    ),
)

classify_prompt = PromptTemplate(
    input_variables=["text"],
    template=(
        "Определи тему текста одним словом.\n"
        "Текст: {text}"
    ),
)

# Собираем цепочку: очистка -> суммаризация -> классификация
clean_chain = clean_prompt | chat | StrOutputParser()
summarize_chain = summarize_prompt | chat | StrOutputParser()
classify_chain = classify_prompt | chat | StrOutputParser()

pipeline = clean_chain | summarize_chain | classify_chain

raw_text = """
Клиент пишет: продукт отличный!!! Но доставка задержалась на 3 дня, и коробка была помята.
В целом доволен, но служба доставки подвела, подвела.
"""

result = pipeline.invoke(raw_text)
print(result)

## Разбор параметров и практические советы

**Что важно понимать:**
- Каждый шаг — отдельный промпт и вызов модели (стоимость суммируется).
- Температуру лучше держать низкой для предсказуемости.
- Если нужен контроль качества — логируйте промежуточные результаты.

**Практика:**
- Для экономии используйте более дешевую модель на ранних шагах.
- Если шаги не зависят друг от друга — можно распараллелить.
- Давайте явные инструкции формата на каждом шаге.